In [0]:
import pandas as pd

import fugue_bigquery.api as fbqa
from fugue_notebook import setup
from statsforecast import StatsForecast
from statsforecast.distributed.fugue import FugueBackend
from statsforecast.models import *

setup(is_lab=True)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/fugue_sql/__init__.py:8: UserWarning: fsql and FugueSQLWorkflow now should be imported directly from fugue, fugue_sql will be removed in 0.9.0
  warnings.warn(
/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  original_result = python_builtin_import(name, globals, locals, fromlist, level)


In [0]:
%%fsql spark
CONNECT bigquery SELECT
    store_number,
    item_description,
    date,
    SUM(bottles_sold) AS total_bottles_sold
FROM
  `bigquery-public-data.iowa_liquor_sales.sales`
WHERE date BETWEEN DATE("2015-01-01") AND DATE("2021-12-24")
GROUP BY store_number, item_description, date

SELECT 
    CONCAT(store_number, '_', item_description) AS unique_id,
    date AS ds,
    total_bottles_sold AS y

PERSIST
YIELD DATAFRAME AS data

/databricks/spark/python/pyspark/sql/pandas/conversion.py:626: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [0]:
sf = StatsForecast(
   models = [ 
       Naive(), 
       MSTL(season_length=7, trend_forecaster=AutoETS(model='ZZN')),
       AutoETS(season_length=7),
       AutoCES(season_length=7),
       SeasonalNaive(season_length=7)
   ], # models to employ, # models to employ
   freq = 'D', # frequency per https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
   fallback_model = Naive()
)
y_pred = sf.forecast(
    df = data.native.repartition(512, 'unique_id'), 
    h = 7
).persist()
y_pred.count()

make fugue
Out[3]: 7459760